In [46]:
# Load Data
import torch
import torch.nn as nn
from torch import optim
import numpy as np
import random
import gensim.downloader as api
import pandas as pd


# The Task: 'Toxicity' Modeling

In 2016 Google published a Kaggle competition called 'toxic comments', offering a prize to the user who could make the best model for predicting whether a given comment was toxic.

https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge

By offering a prize for the best model, google employed a shadow labor force of skilled workers---over 5000 submissions to date. 

kaggle is a good place to look for datasets. 

The data comes from Wikipedia talk pages. 

This dataset is already stored in huggingface: 
    https://huggingface.co/datasets

In [23]:
from datasets import load_dataset

dataset = load_dataset("jigsaw_toxicity_pred", data_dir='../data/jigsaw-toxic-comment-classification-challenge/')


hello


Found cached dataset jigsaw_toxicity_pred (/Users/gabriellachronis/.cache/huggingface/datasets/jigsaw_toxicity_pred/default-e7673256ded46692/1.1.0/9cf096ac4341c35839bc8a9f6a19d93e18e5ad3d84cf05f690d2bc6f7384af85)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 22.89it/s]

hello


Some examples labeled toxic.

# Exploratory Data Analysis
## Let's examine our data.

the data get returned to us as a Dataset object. ie an instance of the huggingface Datasets class. To see what you can do see the documentation here: https://huggingface.co/datasets

in the huggingface datasets library, a split is a specific subset of a dataset like train and test. List a dataset’s split names with the get_dataset_split_names() function. This data is already segmented for us

In [6]:
from datasets import get_dataset_split_names

get_dataset_split_names("jigsaw_toxicity_pred")


['train', 'test']

let's look at the first entry in the train split

In [7]:
# let's look at just train
train = dataset['train']

# Get the first row in the train set. 
train[0] #

{'comment_text': "Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",
 'toxic': 0,
 'severe_toxic': 0,
 'obscene': 0,
 'threat': 0,
 'insult': 0,
 'identity_hate': 0}

Use the - operator to start from the end of the dataset:

In [8]:
# Get the last row in the train set

train[-1]

{'comment_text': '"\nAnd ... I really don\'t think you understand.  I came here and my idea was bad right away.  What kind of community goes ""you have bad ideas"" go away, instead of helping rewrite them.   "',
 'toxic': 0,
 'severe_toxic': 0,
 'obscene': 0,
 'threat': 0,
 'insult': 0,
 'identity_hate': 0}

Indexing by the column name returns a list of all the values in the column:

In [9]:
train["comment_text"][0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

You can combine row and column name indexing to return a specific value at a position:

In [10]:
train[0]["comment_text"]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

But it is important to remember that indexing order matters, especially when working with large audio and image datasets. Indexing by the column name returns all the values in the column first, then loads the value at that position. For large datasets, it may be slower to index by the column name first.

### Aside: dataframes

We are using the built in Datasets class from hugginface, which is useful because it allows us to download a ton of datasets right in Jupyter. But plain old Pandas dataframes are very powerful for storing, examining, manipulating, and transforming data.

Make a dataframe from our Dataset object

In [50]:
df = pd.DataFrame.from_records(train)

In [56]:
df[df['toxic']==1]

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
12,Hey... what is it..\n@ | talk .\nWhat is it......,1,0,0,0,0,0
16,"Bye! \n\nDon't look, come or think of comming ...",1,0,0,0,0,0
42,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1
43,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0
...,...,...,...,...,...,...,...
159494,"""\n\n our previous conversation \n\nyou fuckin...",1,0,1,0,1,1
159514,YOU ARE A MISCHIEVIOUS PUBIC HAIR,1,0,0,0,1,0
159541,Your absurd edits \n\nYour absurd edits on gre...,1,0,1,0,1,0
159546,"""\n\nHey listen don't you ever!!!! Delete my e...",1,0,0,0,1,0


## Multi-label vs Binary classification

Each example is a dictionary with comment text and then 6 annotations, which are coded for either 1 or 0. This is a multilabel classification problem. We are going to simplify it first into a binary classification task. Given a post, is it 'toxic' or not? 

What are the inputs to the model? what is the output?

# Pre-processing the Data

We have to transform our text inputs to numbers. How have we been doing this so far?

<img src="../img/fullyconnected.png" alt="Alternative text" />

Another way to think of the layer of input neurons is as a vector. in fact, this is how we represent the weights in each layer of the network.

How can we represent the comment text as a vector? 



## Loading embeddings

We can use embeddings.

Let's load our word embeddings that we trained on X. 

In [11]:
"""
MISSING
"""

# load word2vec embeddings we trained

'\nMISSING\n'

Alternately, we can use pretrained vectors or embeddings downloaded from the internet. We can use Word2Vec, or GloVe, which is a model that came out a few years later and works very well. 

We use `gensim` which is a great library for working with embeddings and training topic models

In [12]:
info = api.info()

Print out available models (i.e. embeddings)

In [14]:
for model_name, model_data in sorted(info['models'].items()):
    print(
        '%s (%d records): %s' % (
            model_name,
            model_data.get('num_records', -1),
            model_data['description'][:40] + '...',
        )
    )

__testing_word2vec-matrix-synopsis (-1 records): [THIS IS ONLY FOR TESTING] Word vecrors ...
conceptnet-numberbatch-17-06-300 (1917247 records): ConceptNet Numberbatch consists of state...
fasttext-wiki-news-subwords-300 (999999 records): 1 million word vectors trained on Wikipe...
glove-twitter-100 (1193514 records): Pre-trained vectors based on  2B tweets,...
glove-twitter-200 (1193514 records): Pre-trained vectors based on 2B tweets, ...
glove-twitter-25 (1193514 records): Pre-trained vectors based on 2B tweets, ...
glove-twitter-50 (1193514 records): Pre-trained vectors based on 2B tweets, ...
glove-wiki-gigaword-100 (400000 records): Pre-trained vectors based on Wikipedia 2...
glove-wiki-gigaword-200 (400000 records): Pre-trained vectors based on Wikipedia 2...
glove-wiki-gigaword-300 (400000 records): Pre-trained vectors based on Wikipedia 2...
glove-wiki-gigaword-50 (400000 records): Pre-trained vectors based on Wikipedia 2...
word2vec-google-news-300 (3000000 records): Pre-trai

We'll continue with twitter embeddings. Feel free to sub in a different kind and see what kind of results you get.

In [58]:
# download the model and return as object ready for use
embeddings_glove_twitter = api.load("glove-twitter-50")
#embeddings = glove_twitter = api.load("glove-twitter-100")



[==============------------------------------------] 28.0% 55.9/199.5MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[=====================-----------------------------] 43.6% 87.0/199.5MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[===============================-------------------] 62.7% 125.0/199.5MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[===========================================-------] 86.8% 173.1/199.5MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[=================================================-] 99.7% 198.8/199.5MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



We can access the embedding of a single word in the dictionary like this

In [59]:
embeddings['sup']

array([-0.072944 ,  0.31349  , -0.37301  , -0.74591  ,  0.024118 ,
        0.26288  , -0.52766  ,  0.45845  ,  0.66482  , -0.32284  ,
        0.070524 , -0.23753  , -1.9064   , -0.12384  ,  0.34087  ,
        0.10557  ,  0.64763  , -1.5884   ,  0.28275  , -0.48506  ,
        0.37902  , -0.28709  , -0.0066354,  0.24017  , -0.034383 ,
       -0.3468   , -0.0061342, -0.12497  , -0.011999 , -0.63745  ,
       -0.35676  , -0.17062  , -0.86248  ,  0.18034  ,  0.1995   ,
       -0.25941  ,  0.2586   ,  0.17861  ,  0.7617   ,  0.8704   ,
       -0.53839  ,  0.38899  ,  0.27174  ,  0.33564  , -0.29995  ,
        0.9688   ,  0.15263  , -0.48423  ,  0.70449  ,  0.1936   ,
       -0.30026  , -0.46582  , -0.11025  , -1.3443   ,  0.7221   ,
        0.41727  ,  0.078679 , -0.34484  , -0.24705  , -0.65691  ,
       -0.22723  , -0.68642  ,  0.30941  ,  0.38086  ,  0.029261 ,
       -0.22846  , -0.33021  , -0.48214  , -0.28144  , -0.17876  ,
        0.071934 ,  0.17553  ,  0.49776  , -0.44887  ,  0.0209

# Preprocessing

## Transform model inputs into vector representation  (x) 

First we need to transform the input text into a vector representation. 

Let's write a function that constructs an input vector from the training example. It will take a single example in the form of a dictionary OR a comment string, and should have a switch depending on the type of the input. This will help us down the road 

what needs to happen in this function?

In [31]:
def form_input(example, embeddings):
    """
    :example: training example from huggingface dataset in the form of a dictionary, e.g.
    
    {'comment_text': '"\nAnd ... I really don\'t think you understand.  I came here and my idea was bad right away.  What kind of community goes ""you have bad ideas"" go away, instead of helping rewrite them.   "',
     'toxic': 0,
     'severe_toxic': 0,
     'obscene': 0,
     'threat': 0,
     'insult': 0,
     'identity_hate': 0}

    returns a dictionary with a new key, x, which maps to a vector representing the comment
    """
    
    if isinstance(example, str):
        raise Exception("not implemented")
    else:
        raise Exception("not implemented")
        
    example['x'] = None
    return example

We need to process the input string into words. It makes sense to use the same tokenization function we used for the w2v training corpus, because those are the vectors we have to work with.

In [32]:
import re

def tokenize(string):
    """
    tokenization function from n-gram models
    """
    tokenized = re.sub(r'(\w)([.,?!;:])', r'\1 \2', string) 
    tokenized = tokenized.split()
    tokenized = [word.lower() for word in tokenized]
    return tokenized

In [33]:
def form_input(example, word_embeddings):
    """
    :example: training example from huggingface dataset in the form of a dictionary, e.g.
    
    {'comment_text': '"\nAnd ... I really don\'t think you understand.  I came here and my idea was bad right away.  What kind of community goes ""you have bad ideas"" go away, instead of helping rewrite them.   "',
     'toxic': 0,
     'severe_toxic': 0,
     'obscene': 0,
     'threat': 0,
     'insult': 0,
     'identity_hate': 0}

    returns a dictionary with a new key, x, which maps to a vector representing the comment
    """
    
    # get the tokens
    if isinstance(example, str):
        tokens = tokenize(example)
        example = {} # dummy input dict
    else:
        tokens = tokenize(example['comment_text'])

    # get the vectors for each token and average them into one vector
    vecs = []
    for word in tokens:
        try:
            vec = word_embeddings[word]
        except KeyError: # this token is not in our embeddings dictionary
            vec = np.zeros(word_embeddings.vector_size)
        vecs.append(vec)
    
    centroid = np.mean(vecs, axis=0)
    
    # convert numpy array to torch tensor
    torch_tensor = torch.from_numpy(centroid).float()
    
    # return the same example with a new key 'x' containing the vectorized input
    example['x'] = torch_tensor
    return example

We test out our form_input function and see that it adds a new key to the example dict

In [34]:
sample = train[0]
form_input(sample, embeddings)

{'comment_text': "Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",
 'toxic': 0,
 'severe_toxic': 0,
 'obscene': 0,
 'threat': 0,
 'insult': 0,
 'identity_hate': 0,
 'x': tensor([ 1.1892e-01,  1.0474e-01, -7.7170e-02,  2.2719e-02,  2.9379e-02,
         -2.0261e-02,  3.7900e-02, -5.0447e-03,  4.9346e-02, -5.7893e-02,
          6.4910e-02, -6.5754e-02, -3.2642e+00, -7.3882e-02,  7.8788e-02,
         -4.9505e-02,  1.5649e-02, -7.1719e-02, -1.2999e-01, -8.2543e-02,
         -1.6375e-01,  5.9585e-02, -4.1713e-03, -3.5399e-02, -8.7974e-02,
         -2.8266e-01,  5.8411e-02, -2.8793e-02,  2.0931e-01, -1.2959e-01,
         -1.7345e-02,  1.4128e-01, -2.1452e-01,  1.1127e-02,  1.3108e-01,
         -9.3802e-02,  2.5561e-02, -9.2482e-03,  1.0009e-01, -2.3536e-03,
         -3.7532e-01

we can also use a regular string as input

In [35]:
form_input("fuck you you fucking fuck", embeddings)

{'x': tensor([ 0.0953,  0.0600,  0.1371,  0.4924, -0.2468, -0.0166,  0.3260, -0.0351,
          0.0622,  0.4740,  0.1279, -0.0801, -5.2483, -0.2469, -0.1907, -0.0444,
         -0.0053, -0.1239, -0.4338, -0.1554, -0.2208,  0.0270, -0.1308,  0.1786,
          0.6365, -0.7580,  0.0469,  0.2287, -0.0327, -0.3872, -0.4441, -0.2317,
         -0.1610, -0.0405, -0.4412,  0.3505, -0.1863,  0.3733,  0.4390, -0.3021,
         -0.8991,  0.2143, -0.0986, -0.1067,  0.3615, -0.1637,  0.1845,  0.2359,
          0.2526,  0.2617, -0.2349,  0.2826,  0.2627, -0.3209,  0.6101,  0.0562,
         -0.6920,  0.0256, -0.1111,  0.4397,  0.1249,  0.0639, -0.0421,  0.2151,
          0.0659, -0.1925, -0.5575,  0.4118,  0.0263,  0.2235,  0.1402,  0.3073,
          0.1506, -0.0555, -0.2228,  0.0847,  0.4378, -0.2328, -0.4477,  0.2623,
          1.8900,  0.1061, -0.5123, -0.1454,  0.2336,  0.1836, -0.5236, -0.0297,
         -0.2249, -0.1376, -0.2505,  0.0524, -0.1484,  0.4463, -0.1469,  0.5200,
         -0.2752, -0.15

## Preprocessing: Model Outputs

We need to transform the output label into vector representation (gold y). 

### Question: What is our golf 

In [25]:
def form_output(example):
    example['y'] = example['toxic']
    return example

In [26]:
form_output(sample)

{'comment_text': "Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",
 'toxic': 0,
 'severe_toxic': 0,
 'obscene': 0,
 'threat': 0,
 'insult': 0,
 'identity_hate': 0,
 'x': tensor([ 1.1892e-01,  1.0474e-01, -7.7170e-02,  2.2719e-02,  2.9379e-02,
         -2.0261e-02,  3.7900e-02, -5.0447e-03,  4.9346e-02, -5.7893e-02,
          6.4910e-02, -6.5754e-02, -3.2642e+00, -7.3882e-02,  7.8788e-02,
         -4.9505e-02,  1.5649e-02, -7.1719e-02, -1.2999e-01, -8.2543e-02,
         -1.6375e-01,  5.9585e-02, -4.1713e-03, -3.5399e-02, -8.7974e-02,
         -2.8266e-01,  5.8411e-02, -2.8793e-02,  2.0931e-01, -1.2959e-01,
         -1.7345e-02,  1.4128e-01, -2.1452e-01,  1.1127e-02,  1.3108e-01,
         -9.3802e-02,  2.5561e-02, -9.2482e-03,  1.0009e-01, -2.3536e-03,
         -3.7532e-01

# Building the Pytorch Model

We have several decisions to make.

- What is the shape of the input?
- What is the shape of the output?
- What form do we want the output to take? 
- How many layers do we want the model to have?
- What activation function should we use at each layer?
- Do we use dropout?



We subclass nn.Module (which itself is a class and able to keep track of state). In this case, we want to create a class that holds our weights, bias, and method for the forward step. nn.Module has a number of attributes and methods (such as .parameters() and .zero_grad()) which we will be using.

We also can store the loss function here as well.

Here is an example FFNN.


In [28]:
class FFNN(nn.Module):
    """
    Defines the core neural network for doing binary classification over a single datapoint at a time. This consists
    of matrix multiplication, tanh nonlinearity, another matrix multiplication, and then
    a sigmoid layer to give the ouputs.

    The forward() function does the important computation. The backward() method is inherited from nn.Module and
    handles backpropagation.
    """
    def __init__(self, word_embeddings, inp, hid, out):
        """
        Constructs the computation graph by instantiating the various layers and initializing weights.

        :param inp: size of input (integer)
        :param hid: size of hidden layer(integer)
        :param out: size of output (integer), which should be the number of classes
        """
        super(FFNN, self).__init__()
        self.V = nn.Linear(inp, hid)
        self.g = nn.Tanh()
        #self.g = nn.ReLU()
        self.W = nn.Linear(hid, out)
        self.sigmoid = nn.Sigmoid()
        
        # Initialize weights according to a formula due to Xavier Glorot.
        nn.init.xavier_uniform_(self.V.weight)
        nn.init.xavier_uniform_(self.W.weight)
        
        self.num_classes = out
        self.loss = nn.BCELoss()
        

    def forward(self, x):
        """
        Runs the neural network on the given data and returns log probabilities of the various classes.

        :param x: a [inp]-sized tensor of input data
        :return: an [out]-sized tensor of log probabilities. (In general your network can be set up to return either log
        probabilities or a tuple of (loss, log probability) if you want to pass in y to this function as well
        """
        return self.sigmoid(self.W(self.g(self.V(x))))
    

Since we’re now using an object instead of just using a function, we first have to instantiate our model

- how many input dimensions does it have? output?

In [36]:
model = FFNN(embeddings, 100,300,1)
print(model)

FFNN(
  (V): Linear(in_features=100, out_features=300, bias=True)
  (g): Tanh()
  (W): Linear(in_features=300, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (loss): BCELoss()
)


let's run the forward function to compute the output

In [37]:
example = train[0]
print(example)
inp = form_input(example, embeddings)
outp = model.forward(inp['x'])
outp

{'comment_text': "Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27", 'toxic': 0, 'severe_toxic': 0, 'obscene': 0, 'threat': 0, 'insult': 0, 'identity_hate': 0}


tensor([0.4174], grad_fn=<SigmoidBackward0>)

we can access the logits of the returned value

In [38]:
outp.detach()

tensor([0.4174])

And we can change the PyTorch tensor to a Numpy array

In [39]:
outp.detach().numpy()

array([0.41742533], dtype=float32)

# Making Predictions

The final layer is a continuous, differentiable function. It will predict a number between 0 and 1. But we want our output to be either 0 or 1 with no in between. How do we transform this output logit into a prediction?

Write a function called predict() that will generate a prediction that is either a 0 or a 1

In [40]:
def predict(model, word_embeddings, example) -> int:
    x = form_input(example, word_embeddings)['x']
    log_probs = model.forward(x)
    
    if log_probs < 0.5:
        return 0
    else:
        return 1

In [41]:
output = predict(model, embeddings, "fuck you you fucking fuck")
output

0

The model doesn't know what google means by toxicity. we have to train it

# Evaluate

First, a way to evaluate the model performance on a whole bunch of predictions

In [42]:
def print_evaluation(golds, predictions):
    """
    Prints evaluation statistics comparing golds and predictions, each of which is a sequence of 0/1 labels.
    Prints accuracy as well as precision/recall/F1 of the positive class, which can sometimes be informative if either
    the golds or predictions are highly biased.

    :param golds: gold labels, list of ints
    :param predictions: pred labels, list of ints
    :return:
    """
    
    num_correct = 0
    num_pos_correct = 0
    num_pred = 0
    num_gold = 0
    num_total = 0
    if len(golds) != len(predictions):
        raise Exception("Mismatched gold/pred lengths: %i / %i" % (len(golds), len(predictions)))
    for idx in range(0, len(golds)):
        gold = golds[idx]
        #print("gold ", gold)
        prediction = predictions[idx]
        #print("prediction ", prediction)
        if prediction == gold:
            num_correct += 1
        if prediction == 1:
            num_pred += 1
        if gold == 1:
            num_gold += 1
        if prediction == 1 and gold == 1:
            num_pos_correct += 1
        num_total += 1
    acc = float(num_correct) / num_total
    output_str = "Accuracy: %i / %i = %f" % (num_correct, num_total, acc)
    prec = float(num_pos_correct) / num_pred if num_pred > 0 else 0.0
    rec = float(num_pos_correct) / num_gold if num_gold > 0 else 0.0
    f1 = 2 * prec * rec / (prec + rec) if prec > 0 and rec > 0 else 0.0
    output_str += ";\nPrecision (fraction of predicted positives that are correct): %i / %i = %f" % (num_pos_correct, num_pred, prec)
    output_str += ";\nRecall (fraction of true positives predicted correctly): %i / %i = %f" % (num_pos_correct, num_gold, rec)
    output_str += ";\nF1 (harmonic mean of precision and recall): %f;\n" % f1
    print(output_str)
    return acc, f1, output_str

# Lab: Coding the Training Loop

Your task will be to write the training loop for training the feed forward neural network.



### Get the data into the right shape

huggingface datasets has a .map() function that will alter our dataset in place. it asks for a function as its input.
that function has to be of a particular type. in particular, it has to expect something of the same type as a single example (a dictionary) and return a dictionary. We define an anonymous function using a lambda expression that uses our form_input function. no need to worry about the details here unless you are interested because type calculus is sick and so are lambda expressions.


In [43]:
dataset = dataset.map(lambda ex: form_input(ex, embeddings))
dataset['train'][0]

{'comment_text': "Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",
 'toxic': 0,
 'severe_toxic': 0,
 'obscene': 0,
 'threat': 0,
 'insult': 0,
 'identity_hate': 0,
 'x': [0.1189161166548729,
  0.1047351211309433,
  -0.07717016339302063,
  0.022719241678714752,
  0.02937931939959526,
  -0.020260874181985855,
  0.03789982572197914,
  -0.005044739227741957,
  0.049345921725034714,
  -0.05789319425821304,
  0.06490956246852875,
  -0.06575418263673782,
  -3.2642297744750977,
  -0.07388174533843994,
  0.07878798246383667,
  -0.04950515925884247,
  0.015649378299713135,
  -0.07171885669231415,
  -0.12999308109283447,
  -0.08254311978816986,
  -0.1637531965970993,
  0.05958490073680878,
  -0.004171306267380714,
  -0.035398997366428375,
  -0.08797356486320496,
  -0.28265574574470

now our dataset has an extra column with vector representations of each example. we do this once ahead of time rather than on the fly because we will iterate through our training and dev sets many times, and we don't want waste time running form_input over and over

We do the same thing with form_output. clearly, we didnt need separate functions, but it helps to separate our thinking

In [44]:
dataset = dataset.map(lambda ex: form_output(ex))
dataset['train'][0]

{'comment_text': "Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",
 'toxic': 0,
 'severe_toxic': 0,
 'obscene': 0,
 'threat': 0,
 'insult': 0,
 'identity_hate': 0,
 'x': [0.1189161166548729,
  0.1047351211309433,
  -0.07717016339302063,
  0.022719241678714752,
  0.02937931939959526,
  -0.020260874181985855,
  0.03789982572197914,
  -0.005044739227741957,
  0.049345921725034714,
  -0.05789319425821304,
  0.06490956246852875,
  -0.06575418263673782,
  -3.2642297744750977,
  -0.07388174533843994,
  0.07878798246383667,
  -0.04950515925884247,
  0.015649378299713135,
  -0.07171885669231415,
  -0.12999308109283447,
  -0.08254311978816986,
  -0.1637531965970993,
  0.05958490073680878,
  -0.004171306267380714,
  -0.035398997366428375,
  -0.08797356486320496,
  -0.28265574574470

### Dataset Split

Remember, the data are already split into train and test sets. 

It's normal to have a third small data split called val (validate) or dev, that we use to evaluate the model while the training loop is running. Then, at the end of training, we evaluate on the test set. The dev set can be used to do things like choose **hyperparameters** for the model---we can't use the test set for this because then we'd be 'cheating'

In [ ]:
# load training data

train = dataset['train']
dev = dataset['test']
test = dataset['test']

print(repr(len(train)) + " / " + repr(len(dev)) + " / " + repr(len(test)) + " train/dev/test examples")


Let's set the model hyperparameters. 

In [ ]:
# set hyperparameters

num_epochs = 10
hidden_size = 200
initial_learning_rate = 0.1

We initialize a model and an optimizer

In [ ]:
ffnn = FFNN(embeddings, embeddings.vector_size, hidden_size, 1)
optimizer = optim.Adam(ffnn.parameters(), lr=initial_learning_rate)

Next we need to code the actual training loop. This loop will run through the training data for a set number of epochs. In each epoch, it should iterate through all of the training examples. Ideally, it will do this in a random order every time. You should run the example through the model, calculate the loss, and then use the loss to run the backward step. **be sure to zero out the gradients before calling the forward step!!** This is done with `ffnn.zero_grad()` Our loss function is stored in the FFNN object, and can be called using `ffnn.loss()`. The backpropagation step is run on the loss, i.e. `loss.backward()` (where `loss` is the loss computed at this time step). At the end of each epoch, the

In [ ]:
## training loop for toxicity classification



for epoch in range(0, num_epochs):
    ex_indices = [i for i in range(0, len(train))]
    random.shuffle(ex_indices)
    total_loss = 0.0
    
    ffnn.train()
    for idx in ex_indices:
        
        x = torch.Tensor(train[idx]['x'])
        
        #print(x)
        y = train[idx]['y']
        # transform to torch
        y = torch.from_numpy(np.asarray(y,dtype=np.float32)).unsqueeze(-1)
        
        # Build one-hot representation of y. Instead of the label 0 or 1, y_onehot is either [0, 1] or [1, 0]. This
        # way we can take the dot product directly with a probability vector to get class probabilities.
        #y_onehot = torch.zeros(ffnn.num_classes)
        
        # scatter will write the value of 1 into the position of y_onehot given by y
        #y_onehot.scatter_(0, torch.from_numpy(np.asarray(y,dtype=np.int64)), 1)
        
        # Zero out the gradients from the FFNN object. *THIS IS VERY IMPORTANT TO DO BEFORE CALLING BACKWARD()*
        ffnn.zero_grad()
        y_hat = ffnn.forward(x)
        
        # Can also use built-in NLLLoss as a shortcut here but we're being explicit here
        loss = ffnn.loss(y_hat, y)
        total_loss += loss
        
        # Computes the gradient and takes the optimizer step
        loss.backward()
        optimizer.step()
    print("Total loss on epoch %s: %f" % (epoch, total_loss))
    
    ffnn.eval()
    dev_y_hats = [predict(ffnn, embeddings, ex['comment_text']) for ex in dev]
    print_evaluation(dev['y'], dev_y_hats)



# Make Predictions

Let's use the model to make some predictions:

In [ ]:
predict(ffnn, embeddings, "fuck you you fucking fuck")

# Improving the Model with Features

This works okay. How can we get better performance? See if you can improve the model to X F1 score. You could try using different embeddings. You could change the model itself, adding more layers or trying a different activation function. You could add dropout after one of the layers (this takes one line to implement with pytorch, but you might have to look up about what it is or how it's used). You can also add hand-crafted features in addition to the embeddings to serve as input. This is a very common strategy and usually helps a lot.

Instead of just representing the input as word embeddings, we can represent the input as a vector of word embeddings concatenated with additional dimensions that encode other variables we are interested in. The features can be real-valued, like: how many tokens are in the example? what's the average word length? Features can also be binary, like: does the sentence contain a question mark? 

For example, s length. Here is a new form_input that adds an average word length feature and a contains X feature

In [ ]:
def extract_features(tokens):
    question = 1 if True else 0
    features = {
        "avg_word_length": np.mean([len(t) for t in tokens]),
        "contains_q": 1 if "?" in tokens else 0
    }
    return features

def form_input_updated(example, word_embeddings):
    """
    :example: training example from huggingface dataset in the form of a dictionary, e.g.
    
    {'comment_text': '"\nAnd ... I really don\'t think you understand.  I came here and my idea was bad right away.  What kind of community goes ""you have bad ideas"" go away, instead of helping rewrite them.   "',
     'toxic': 0,
     'severe_toxic': 0,
     'obscene': 0,
     'threat': 0,
     'insult': 0,
     'identity_hate': 0}

    returns a dictionary with a new key, x, which maps to a vector representing the comment
    """
    
    if isinstance(example, str):
        tokens = tokenize(example)
        example = {} # dummy input dict
    else:
        tokens = tokenize(example['comment_text'])

    vecs = []
    for word in tokens:
        try:
            vec = word_embeddings[word]
        except KeyError: # this token is not in our embeddings dictionary
            vec = np.zeros(word_embeddings.vector_size)
        vecs.append(vec)
    
    centroid = np.mean(vecs, axis=0)
    
    
    features_dict = extract_features(tokens)
    features_list = list(features_dict.values())
    
    final_vector = np.append(centroid, features_list, axis=0)
    
    # we need torch form which is a tensor, not a numpy array
    
    torch_tensor = torch.from_numpy(final_vector).float()
    example['x'] = torch_tensor
    return example
    

In [ ]:
form_input_updated("hello my name is fhwfhjwhfjwklfjwj4klwfkl?", embeddings)

In [ ]:
# re-form our inputs, make a new model and train that one. 

ffnn = FFNN(embeddings, embeddings.vector_size, hidden_size, 1)
optimizer = optim.Adam(ffnn.parameters(), lr=initial_learning_rate)

dataset = dataset.map(lambda ex: form_input_updated(ex, embeddings))


In [ ]:
dataset['train'][0]["x"]

In [ ]:
## training loop for toxicity classification



for epoch in range(0, num_epochs):
    ex_indices = [i for i in range(0, len(train))]
    random.shuffle(ex_indices)
    total_loss = 0.0
    
    ffnn.train()
    for idx in ex_indices:
        
        x = torch.Tensor(train[idx]['x'])
        
        #print(x)
        y = train[idx]['y']
        # transform to torch
        y = torch.from_numpy(np.asarray(y,dtype=np.float32)).unsqueeze(-1)
        
        # Build one-hot representation of y. Instead of the label 0 or 1, y_onehot is either [0, 1] or [1, 0]. This
        # way we can take the dot product directly with a probability vector to get class probabilities.
        #y_onehot = torch.zeros(ffnn.num_classes)
        
        # scatter will write the value of 1 into the position of y_onehot given by y
        #y_onehot.scatter_(0, torch.from_numpy(np.asarray(y,dtype=np.int64)), 1)
        
        # Zero out the gradients from the FFNN object. *THIS IS VERY IMPORTANT TO DO BEFORE CALLING BACKWARD()*
        ffnn.zero_grad()
        y_hat = ffnn.forward(x)
        
        # Can also use built-in NLLLoss as a shortcut here but we're being explicit here
        loss = ffnn.loss(y_hat, y)
        total_loss += loss
        
        # Computes the gradient and takes the optimizer step
        loss.backward()
        optimizer.step()
    print("Total loss on epoch %s: %f" % (epoch, total_loss))
    
    ffnn.eval()
    dev_y_hats = [predict(ffnn, embeddings, ex['comment_text']) for ex in dev]
    print_evaluation(dev['y'], dev_y_hats)



# Appendix: Example Training Loop

Here is an example training loop for learning the XOR function

In [ ]:
# MAKE THE DATA
# Synthetic data for XOR: y = x1 XOR x2
train_xs = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
train_ys = np.array([0, 1, 1, 0], dtype=np.float32)

# Define some constants
# Inputs are of size 2
feat_vec_size = 2

# Let's use 4 hidden units
embedding_size = 4

# We're using 2 classes. What's presented here is multi-class code that can scale to more classes, though
# slightly more compact code for the binary case is possible.
num_classes = 1


# set hyperparameters
num_epochs = 100
ffnn = FFNN(feat_vec_size, embedding_size, num_classes)
initial_learning_rate = 0.1
optimizer = optim.Adam(ffnn.parameters(), lr=initial_learning_rate)


# RUN TRAINING
for epoch in range(0, num_epochs):
    
    ex_indices = [i for i in range(0, len(train_xs))]
    random.shuffle(ex_indices)
    total_loss = 0.0

    for idx in ex_indices:
        x =  torch.from_numpy(train_xs[idx]).float()
        y = train_ys[idx]
        y = torch.from_numpy(np.asarray(y,dtype=np.float32)).unsqueeze(-1)

        # Build one-hot representation of y. Instead of the label 0 or 1, y_onehot is either [0, 1] or [1, 0]. This
        # way we can take the dot product directly with a probability vector to get class probabilities.
        #y_onehot = torch.zeros(num_classes)
        
        # scatter will write the value of 1 into the position of y_onehot given by y
        #y_onehot.scatter_(0, torch.from_numpy(np.asarray(y,dtype=np.int64)), 1)
        # Zero out the gradients from the FFNN object. *THIS IS VERY IMPORTANT TO DO BEFORE CALLING BACKWARD()*
        ffnn.zero_grad()
        log_probs = ffnn.forward(x)
        
        print(x)
        print(log_probs)
        print(y)
        
        # Can also use built-in NLLLoss as a shortcut here but we're being explicit here
        loss = ffnn.loss(log_probs, y)
        
        total_loss += loss
        # Computes the gradient and takes the optimizer step
        loss.backward()
        optimizer.step()
    print("Total loss on epoch %i: %f" % (epoch, total_loss))
    

# Evaluate on the train set
# RUN TRAINING AND TEST

train_correct = 0
for idx in range(0, len(train_xs)):
    x = torch.from_numpy(train_xs[idx]).float()
    y = train_ys[idx]
    log_probs = ffnn.forward(x)
    prediction = 1 if log_probs > 0.5 else 0
    if y == prediction:
        train_correct += 1
    print("Example " + repr(train_xs[idx]) + "; gold = " + repr(train_ys[idx]) + "; pred = " +\
          repr(prediction) + " with probs " + repr(log_probs))
print(repr(train_correct) + "/" + repr(len(train_ys)) + " correct after training")

# References

- https://vijaygadepalli.medium.com/toxic-comments-classification-696603741872
    